In [49]:
import tensorflow as tf
from tflite2xcore import utils, analyze, version
import tflite2xcore.converter as xcore_conv
import numpy as np

# Make a Model to convert
Use Keras to make a model of arbiraty size and shape

In [50]:
pool_size = (2, 2)
input_shape = (3, 3, 4)
model = tf.keras.Sequential([
    tf.keras.layers.AveragePooling2D(pool_size=pool_size, input_shape=input_shape)
])
# is this necessary?
model.compile()

## Convert keras model into a tflite model
The xcore converter cannot optimise a keras model to run on xcore devices, so it must first be converted into a tflite file.

In [51]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

### Representitive Dataset

Tensorflow can optimise the converted model if you pass it a a representative dataset. This dataset can be a small subset (around ~100-500 samples) of the training or validation data

The below function randomly gemerates this, but see [the tensorflow ducumentation](https://www.tensorflow.org/lite/performance/post_training_quantization) to see how to do this in practice.

In [52]:
# As an example use a random dataset
def representative_dataset():
    batch_size = 8
    for _ in range(100):
      data = np.random.uniform(-0.1, 0.001, (batch_size, *input_shape))
      yield [data.astype(np.float32)]

* **tf.lite.Optimize.DEFAULT:** Default optimization strategy that quantizes model weights. Enhanced optimizations are gained by providing a representative dataset that quantizes biases and activations as well. Converter will do its best to reduce size and latency, while minimizing the loss in accuracy.

* **target_spec.supported_ops:** Import TFLITE ops. [Tensorflow docs](https://www.tensorflow.org/lite/guide/ops_select)

In [53]:
# Set up the converter to convert our float model into int8 quantised model
#explain  https://www.tensorflow.org/lite/performance/post_training_quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8 
converter.inference_output_type = tf.int8

tflite_model = converter.convert()

# Save the model.
tflite_model_path = 'avgpooling2d.tflite'
with open(tflite_model_path, 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/fg/_pf9q2tj3cl9yfjb392zl7rm0000gn/T/tmpy3xwidz9/assets


INFO:tensorflow:Assets written to: /var/folders/fg/_pf9q2tj3cl9yfjb392zl7rm0000gn/T/tmpy3xwidz9/assets
2021-10-18 01:46:23.746990: I tensorflow/core/grappler/devices.cc:78] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2021-10-18 01:46:23.747082: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2021-10-18 01:46:23.747987: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1144] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.004ms.
  function_optimizer: function_optimizer did nothing. time = 0ms.

2021-10-18 01:46:23.761489: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:345] Ignored output_format.
2021-10-18 01:46:23.761515: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:348] Ignored drop_control_dependency.
fully_quantize: 0, inference_type: 6, inpu

# Optimise model for XCore
Use `xcore_conv.convert(input_path, output_path)` to make an xcore optimised version of the model.

In [54]:
xcore_optimised_path = 'xcore_model.tflite'
xcore_conv.convert(tflite_model_path, xcore_optimised_path)

# Check it worked
To check if it worked, we can use the interpreters to run the models and make sure that they produce the same output.

For normal tensorflow tflite models, use `tensorflow.lite.Interpreter`. For XCore optimised models, the `XCOREInterpreter` must be used.

In [55]:
from xcore_interpreters import XCOREInterpreter

In [56]:
tf_interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
tf_interpreter.allocate_tensors()

tf_input_details = tf_interpreter.get_input_details()
tf_output_details = tf_interpreter.get_output_details()

tf_input_shape = tf_input_details[0]['shape']
# Fill with 126 so that xcore can be given same input
tf_input_data = np.array(np.random.randint(126, 127, tf_input_shape), dtype=np.int8)

tf_interpreter.set_tensor(tf_input_details[0]['index'], tf_input_data)

tf_interpreter.invoke()
tf_output_data = tf_interpreter.get_tensor(tf_output_details[0]['index'])

In [60]:
xcore_interpreter = XCOREInterpreter(model_path=xcore_optimised_path)
xcore_interpreter.allocate_tensors()

xcore_input_details = xcore_interpreter.get_input_details()
xcore_output_details = xcore_interpreter.get_output_details()

xcore_input_shape = xcore_input_details[0]['shape']
# Fill with 126 so that xcore converter has the same inputs
xcore_input_data = np.array(np.random.randint(126, 127, xcore_input_shape), dtype=np.int8)

xcore_interpreter.set_tensor(xcore_input_details[0]['index'], xcore_input_data)

xcore_interpreter.invoke()
xcore_output_data = xcore_interpreter.get_tensor(xcore_output_details[0]['index'])

In [59]:
print("Both outputs equal:")
print(np.array_equal(xcore_output_data[0], tf_output_data[0]))


Both outputs equal:
True
